In [92]:
import os

import pandas as pd

raw_data_dir = r"C:\Users\Andrew\Documents\UofT2016\Bike App Data\Bike Data Original"
cut_data_dir = r"C:\Users\Andrew\Documents\UofT2016\Speed Analysis\toronto-cycling-speed-analysis\Data\Cut Data"

In [93]:
raw_data = [os.path.join(raw_data_dir, f) for f in os.listdir(raw_data_dir) if "coords" in f]
trip_surveys = pd.read_csv(os.path.join(raw_data_dir, "CyclingApp_Trip_Surveys.csv"))
user_surveys = pd.read_csv(os.path.join(raw_data_dir, "CyclingApp_User_Surveys.csv"), 
                           encoding = "ISO-8859-1")
compiled_data = pd.merge(trip_surveys, user_surveys, how="inner", on="app_user_id")

In [94]:
for day in raw_data:
    data = pd.read_csv(day, encoding = "ISO-8859-1")
    trip_data = pd.merge(data, compiled_data, how="inner", on="trip_id")
    trip_data.loc[trip_data["speed"] < 0, "speed"] = 0
    
    for trip_id in trip_data["trip_id"].unique():
        trip = trip_data[trip_data["trip_id"] == trip_id]
        trip = trip.sort_values("recorded_at")
        trip_avg_speed = trip["speed"].mean()
        
        start_cut = 0
        end_cut = 0
        finished_start_cut = False
        for i, speed in enumerate((speed for i, speed in trip["speed"].iteritems())):
            if speed > trip_avg_speed / 5.0 and not finished_start_cut:
                finished_start_cut = True
                start_cut = i
            if speed > trip_avg_speed:
                end_cut = i + 1
        cut_trip = trip.iloc[start_cut : end_cut]
        
        if cut_trip["speed"].mean() < 0.5 or len(cut_trip.index) < 10: # This is 1.8 km/h, a very slow walking speed
            continue
        cut_trip.to_csv(os.path.join(cut_data_dir, "%d.csv" % trip_id), 
                        index_label="ObjectID")
        try:
            od_trip = cut_trip.iloc[[0, len(cut_trip) - 1]]
        except:
            print(trip_id)
            raise
        od_trip.to_csv(os.path.join(cut_data_dir, "%d_od.csv" % trip_id),
                      index=False)